In [1]:
import sys
import enlighten
import pandas as pd
from os.path import join, exists

In [2]:
# assembling a dictionary of data structure shortnames and columns of interest
variables = {
    "abcd_smrip10201": [
        "imgincl_dmri_include",
        "imgincl_rsfmri_include",
        "imgincl_t1w_include",
        "imgincl_t2w_include",
        "interview_age",
        "smri_vol_scs_intracranialv",
        "smri_vol_cdk_total",
        "mrisdp_604",
        "smri_area_cdk_banksstslh",
        "smri_area_cdk_cdacatelh",
        "smri_area_cdk_cdmdfrlh",
        "smri_area_cdk_cuneuslh",
        "smri_area_cdk_ehinallh",
        "smri_area_cdk_fusiformlh",
        "smri_area_cdk_ifpllh",
        "smri_area_cdk_iftmlh",
        "smri_area_cdk_ihcatelh",
        "smri_area_cdk_locclh",
        "smri_area_cdk_lobfrlh",
        "smri_area_cdk_linguallh",
        "smri_area_cdk_mobfrlh",
        "smri_area_cdk_mdtmlh",
        "smri_area_cdk_parahpallh",
        "smri_area_cdk_paracnlh",
        "smri_area_cdk_parsopclh",
        "smri_area_cdk_parsobislh",
        "smri_area_cdk_parstgrislh",
        "smri_area_cdk_pericclh",
        "smri_area_cdk_postcnlh",
        "smri_area_cdk_ptcatelh",
        "smri_area_cdk_precnlh",
        "smri_area_cdk_pclh",
        "smri_area_cdk_rracatelh",
        "smri_area_cdk_rrmdfrlh",
        "smri_area_cdk_sufrlh",
        "smri_area_cdk_supllh",
        "smri_area_cdk_sutmlh",
        "smri_area_cdk_smlh",
        "smri_area_cdk_frpolelh",
        "smri_area_cdk_tmpolelh",
        "smri_area_cdk_trvtmlh",
        "smri_area_cdk_insulalh",
        "smri_area_cdk_banksstsrh",
        "smri_area_cdk_cdacaterh",
        "smri_area_cdk_cdmdfrrh",
        "smri_area_cdk_cuneusrh",
        "smri_area_cdk_ehinalrh",
        "smri_area_cdk_fusiformrh",
        "smri_area_cdk_ifplrh",
        "smri_area_cdk_iftmrh",
        "smri_area_cdk_ihcaterh",
        "smri_area_cdk_loccrh",
        "smri_area_cdk_lobfrrh",
        "smri_area_cdk_lingualrh",
        "smri_area_cdk_mobfrrh",
        "smri_area_cdk_mdtmrh",
        "smri_area_cdk_parahpalrh",
        "smri_area_cdk_paracnrh",
        "smri_area_cdk_parsopcrh",
        "smri_area_cdk_parsobisrh",
        "smri_area_cdk_parstgrisrh",
        "smri_area_cdk_periccrh",
        "smri_area_cdk_postcnrh",
        "smri_area_cdk_ptcaterh",
        "smri_area_cdk_precnrh",
        "smri_area_cdk_pcrh",
        "smri_area_cdk_rracaterh",
        "smri_area_cdk_rrmdfrrh",
        "smri_area_cdk_sufrrh",
        "smri_area_cdk_suplrh",
        "smri_area_cdk_sutmrh",
        "smri_area_cdk_smrh",
        "smri_area_cdk_frpolerh",
        "smri_area_cdk_tmpolerh",
        "smri_area_cdk_trvtmrh",
        "smri_area_cdk_insularh",
        #"smri_area_cf12_cnlh",
        #"smri_area_cf12_occlh",
        #"smri_area_cf12_ptoltmlh",
        #"smri_area_cf12_supllh",
        #"smri_area_cf12_obfrlh",
        #"smri_area_cf12_sutmlh",
        #"smri_area_cf12_ifpllh",
        #"smri_area_cf12_dmfrlh",
        #"smri_area_cf12_aomtmlh",
        #"smri_area_cf12_pclh",
        #"smri_area_cf12_dlprefrlh",
        #"smri_area_cf12_parsopclh",
        #"smri_area_cf12_cnrh",
        #"smri_area_cf12_occrh",
        #"smri_area_cf12_ptoltmrh",
        #"smri_area_cf12_suplrh",
        #"smri_area_cf12_obfrrh",
        #"smri_area_cf12_sutmrh",
        #"smri_area_cf12_ifplrh",
        #"smri_area_cf12_dmfrrh",
        #"smri_area_cf12_aomtmrh",
        #"smri_area_cf12_pcrh",
        #"smri_area_cf12_dlprefrrh",
        #"smri_area_cf12_parsopcrh",
        "smri_thick_cdk_banksstslh",
        "smri_thick_cdk_cdacatelh",
        "smri_thick_cdk_cdmdfrlh",
        "smri_thick_cdk_cuneuslh",
        "smri_thick_cdk_ehinallh",
        "smri_thick_cdk_fusiformlh",
        "smri_thick_cdk_ifpllh",
        "smri_thick_cdk_iftmlh",
        "smri_thick_cdk_ihcatelh",
        "smri_thick_cdk_locclh",
        "smri_thick_cdk_lobfrlh",
        "smri_thick_cdk_linguallh",
        "smri_thick_cdk_mobfrlh",
        "smri_thick_cdk_mdtmlh",
        "smri_thick_cdk_parahpallh",
        "smri_thick_cdk_paracnlh",
        "smri_thick_cdk_parsopclh",
        "smri_thick_cdk_parsobislh",
        "smri_thick_cdk_parstgrislh",
        "smri_thick_cdk_pericclh",
        "smri_thick_cdk_postcnlh",
        "smri_thick_cdk_ptcatelh",
        "smri_thick_cdk_precnlh",
        "smri_thick_cdk_pclh",
        "smri_thick_cdk_rracatelh",
        "smri_thick_cdk_rrmdfrlh",
        "smri_thick_cdk_sufrlh",
        "smri_thick_cdk_supllh",
        "smri_thick_cdk_sutmlh",
        "smri_thick_cdk_smlh",
        "smri_thick_cdk_frpolelh",
        "smri_thick_cdk_tmpolelh",
        "smri_thick_cdk_trvtmlh",
        "smri_thick_cdk_insulalh",
        "smri_thick_cdk_banksstsrh",
        "smri_thick_cdk_cdacaterh",
        "smri_thick_cdk_cdmdfrrh",
        "smri_thick_cdk_cuneusrh",
        "smri_thick_cdk_ehinalrh",
        "smri_thick_cdk_fusiformrh",
        "smri_thick_cdk_ifplrh",
        "smri_thick_cdk_iftmrh",
        "smri_thick_cdk_ihcaterh",
        "smri_thick_cdk_loccrh",
        "smri_thick_cdk_lobfrrh",
        "smri_thick_cdk_lingualrh",
        "smri_thick_cdk_mobfrrh",
        "smri_thick_cdk_mdtmrh",
        "smri_thick_cdk_parahpalrh",
        "smri_thick_cdk_paracnrh",
        "smri_thick_cdk_parsopcrh",
        "smri_thick_cdk_parsobisrh",
        "smri_thick_cdk_parstgrisrh",
        "smri_thick_cdk_periccrh",
        "smri_thick_cdk_postcnrh",
        "smri_thick_cdk_ptcaterh",
        "smri_thick_cdk_precnrh",
        "smri_thick_cdk_pcrh",
        "smri_thick_cdk_rracaterh",
        "smri_thick_cdk_rrmdfrrh",
        "smri_thick_cdk_sufrrh",
        "smri_thick_cdk_suplrh",
        "smri_thick_cdk_sutmrh",
        "smri_thick_cdk_smrh",
        "smri_thick_cdk_frpolerh",
        "smri_thick_cdk_tmpolerh",
        "smri_thick_cdk_trvtmrh",
        "smri_thick_cdk_insularh",
        #"smri_thick_cf12_cnlh",
        #"smri_thick_cf12_occlh",
        #"smri_thick_cf12_ptoltmlh",
        #"smri_thick_cf12_supllh",
        #"smri_thick_cf12_obfrlh",
        #"smri_thick_cf12_sutmlh",
        #"smri_thick_cf12_ifpllh",
        #"smri_thick_cf12_dmfrlh",
        #"smri_thick_cf12_aomtmlh",
        #"smri_thick_cf12_pclh",
        #"smri_thick_cf12_dlprefrlh",
        #"smri_thick_cf12_parsopclh",
        #"smri_thick_cf12_cnrh",
        #"smri_thick_cf12_occrh",
        #"smri_thick_cf12_ptoltmrh",
        #"smri_thick_cf12_suplrh",
        #"smri_thick_cf12_obfrrh",
        #"smri_thick_cf12_sutmrh",
        #"smri_thick_cf12_ifplrh",
        #"smri_thick_cf12_dmfrrh",
        #"smri_thick_cf12_aomtmrh",
        #"smri_thick_cf12_pcrh",
        #"smri_thick_cf12_dlprefrrh",
        #"smri_thick_cf12_parsopcrh",
        "smri_vol_scs_crbwmatterlh",
        "smri_vol_scs_crbcortexlh",
        "smri_vol_scs_tplh",
        "smri_vol_scs_caudatelh",
        "smri_vol_scs_putamenlh",
        "smri_vol_scs_pallidumlh",
        "smri_vol_scs_bstem",
        "smri_vol_scs_hpuslh",
        "smri_vol_scs_amygdalalh",
        "smri_vol_scs_aal",
        "smri_vol_scs_vedclh",
        "smri_vol_scs_crbwmatterrh",
        "smri_vol_scs_crbcortexrh",
        "smri_vol_scs_tprh",
        "smri_vol_scs_caudaterh",
        "smri_vol_scs_putamenrh",
        "smri_vol_scs_pallidumrh",
        "smri_vol_scs_hpusrh",
        "smri_vol_scs_amygdalarh",
        "smri_vol_scs_aar",
        "smri_vol_scs_vedcrh",
    ],
    "abcd_smrip20201": [
        #"smri_t1wcnt_cf12_cnlh",
        #"smri_t1wcnt_cf12_occlh",
        #"smri_t1wcnt_cf12_ptoltmlh",
        #"smri_t1wcnt_cf12_supllh",
        #"smri_t1wcnt_cf12_obfrlh",
        #"smri_t1wcnt_cf12_sutmlh",
        #"smri_t1wcnt_cf12_ifpllh",
        #"smri_t1wcnt_cf12_dmfrlh",
        #"smri_t1wcnt_cf12_aomtmlh",
        #"smri_t1wcnt_cf12_pclh",
        #"smri_t1wcnt_cf12_dlprefrlh",
        #"smri_t1wcnt_cf12_parsopclh",
        #"smri_t1wcnt_cf12_cnrh",
        #"smri_t1wcnt_cf12_occrh",
        #"smri_t1wcnt_cf12_ptoltmrh",
        #"smri_t1wcnt_cf12_suplrh",
        #"smri_t1wcnt_cf12_obfrrh",
        #"smri_t1wcnt_cf12_sutmrh",
        #"smri_t1wcnt_cf12_ifplrh",
        #"smri_t1wcnt_cf12_dmfrrh",
        #"smri_t1wcnt_cf12_aomtmrh",
        #"smri_t1wcnt_cf12_pcrh",
        #"smri_t1wcnt_cf12_dlprefrrh",
        #"smri_t1wcnt_cf12_parsopcrh",
        "smri_t1wcnt_cdk_banksstslh",
        "smri_t1wcnt_cdk_cdacatelh",
        "smri_t1wcnt_cdk_cdmdfrlh",
        "smri_t1wcnt_cdk_cuneuslh",
        "smri_t1wcnt_cdk_ehinallh",
        "smri_t1wcnt_cdk_fusiformlh",
        "smri_t1wcnt_cdk_ifpllh",
        "smri_t1wcnt_cdk_iftmlh",
        "smri_t1wcnt_cdk_ihcatelh",
        "smri_t1wcnt_cdk_locclh",
        "smri_t1wcnt_cdk_lobfrlh",
        "smri_t1wcnt_cdk_linguallh",
        "smri_t1wcnt_cdk_mobfrlh",
        "smri_t1wcnt_cdk_mdtmlh",
        "smri_t1wcnt_cdk_parahpallh",
        "smri_t1wcnt_cdk_paracnlh",
        "smri_t1wcnt_cdk_parsopclh",
        "smri_t1wcnt_cdk_parsobislh",
        "smri_t1wcnt_cdk_parstgrislh",
        "smri_t1wcnt_cdk_pericclh",
        "smri_t1wcnt_cdk_postcnlh",
        "smri_t1wcnt_cdk_ptcatelh",
        "smri_t1wcnt_cdk_precnlh",
        "smri_t1wcnt_cdk_pclh",
        "smri_t1wcnt_cdk_rracatelh",
        "smri_t1wcnt_cdk_rrmdfrlh",
        "smri_t1wcnt_cdk_sufrlh",
        "smri_t1wcnt_cdk_supllh",
        "smri_t1wcnt_cdk_sutmlh",
        "smri_t1wcnt_cdk_smlh",
        "smri_t1wcnt_cdk_frpolelh",
        "smri_t1wcnt_cdk_tmpolelh",
        "smri_t1wcnt_cdk_trvtmlh",
        "smri_t1wcnt_cdk_insulalh",
        "smri_t1wcnt_cdk_banksstsrh",
        "smri_t1wcnt_cdk_cdacaterh",
        "smri_t1wcnt_cdk_cdmdfrrh",
        "smri_t1wcnt_cdk_cuneusrh",
        "smri_t1wcnt_cdk_ehinalrh",
        "smri_t1wcnt_cdk_fusiformrh",
        "smri_t1wcnt_cdk_ifplrh",
        "smri_t1wcnt_cdk_iftmrh",
        "smri_t1wcnt_cdk_ihcaterh",
        "smri_t1wcnt_cdk_loccrh",
        "smri_t1wcnt_cdk_lobfrrh",
        "smri_t1wcnt_cdk_lingualrh",
        "smri_t1wcnt_cdk_mobfrrh",
        "smri_t1wcnt_cdk_mdtmrh",
        "smri_t1wcnt_cdk_parahpalrh",
        "smri_t1wcnt_cdk_paracnrh",
        "smri_t1wcnt_cdk_parsopcrh",
        "smri_t1wcnt_cdk_parsobisrh",
        "smri_t1wcnt_cdk_parstgrisrh",
        "smri_t1wcnt_cdk_periccrh",
        "smri_t1wcnt_cdk_postcnrh",
        "smri_t1wcnt_cdk_ptcaterh",
        "smri_t1wcnt_cdk_precnrh",
        "smri_t1wcnt_cdk_pcrh",
        "smri_t1wcnt_cdk_rracaterh",
        "smri_t1wcnt_cdk_rrmdfrrh",
        "smri_t1wcnt_cdk_sufrrh",
        "smri_t1wcnt_cdk_suplrh",
        "smri_t1wcnt_cdk_sutmrh",
        "smri_t1wcnt_cdk_smrh",
        "smri_t1wcnt_cdk_frpolerh",
        "smri_t1wcnt_cdk_tmpolerh",
        "smri_t1wcnt_cdk_trvtmrh",
        "smri_t1wcnt_cdk_insularh",
        "smri_t1w_scs_crbwmatterlh",
        "smri_t1w_scs_crbcortexlh",
        "smri_t1w_scs_tplh",
        "smri_t1w_scs_caudatelh",
        "smri_t1w_scs_putamenlh",
        "smri_t1w_scs_pallidumlh",
        "smri_t1w_scs_bstem",
        "smri_t1w_scs_hpuslh",
        "smri_t1w_scs_amygdalalh",
        "smri_t1w_scs_aal",
        "smri_t1w_scs_vedclh",
        "smri_t1w_scs_crbwmatterrh",
        "smri_t1w_scs_crbcortexrh",
        "smri_t1w_scs_tprh",
        "smri_t1w_scs_caudaterh",
        "smri_t1w_scs_putamenrh",
        "smri_t1w_scs_pallidumrh",
        "smri_t1w_scs_hpusrh",
        "smri_t1w_scs_amygdalarh",
        "smri_t1w_scs_aar",
        "smri_t1w_scs_vedcrh",
    ],
    "abcd_smrip30201": [
        "smri_t2w_scs_crbwmatterlh",
        "smri_t2w_scs_crbcortexlh",
        "smri_t2w_scs_tplh",
        "smri_t2w_scs_caudatelh",
        "smri_t2w_scs_putamenlh",
        "smri_t2w_scs_pallidumlh",
        "smri_t2w_scs_bstem",
        "smri_t2w_scs_hpuslh",
        "smri_t2w_scs_amygdalalh",
        "smri_t2w_scs_aal",
        "smri_t2w_scs_vedclh",
        "smri_t2w_scs_crbwmatterrh",
        "smri_t2w_scs_crbcortexrh",
        "smri_t2w_scs_tprh",
        "smri_t2w_scs_caudaterh",
        "smri_t2w_scs_putamenrh",
        "smri_t2w_scs_pallidumrh",
        "smri_t2w_scs_hpusrh",
        "smri_t2w_scs_amygdalarh",
        "smri_t2w_scs_aar",
        "smri_t2w_scs_vedcrh",
    ],
    "abcd_mrisdp10201": [
        "mrisdp_1",
        "mrisdp_2",
        "mrisdp_3",
        "mrisdp_4",
        "mrisdp_5",
        "mrisdp_6",
        "mrisdp_7",
        "mrisdp_8",
        "mrisdp_9",
        "mrisdp_10",
        "mrisdp_11",
        "mrisdp_12",
        "mrisdp_13",
        "mrisdp_14",
        "mrisdp_15",
        "mrisdp_16",
        "mrisdp_17",
        "mrisdp_18",
        "mrisdp_19",
        "mrisdp_20",
        "mrisdp_21",
        "mrisdp_22",
        "mrisdp_23",
        "mrisdp_24",
        "mrisdp_25",
        "mrisdp_26",
        "mrisdp_27",
        "mrisdp_28",
        "mrisdp_29",
        "mrisdp_30",
        "mrisdp_31",
        "mrisdp_32",
        "mrisdp_33",
        "mrisdp_34",
        "mrisdp_35",
        "mrisdp_36",
        "mrisdp_37",
        "mrisdp_38",
        "mrisdp_39",
        "mrisdp_40",
        "mrisdp_41",
        "mrisdp_42",
        "mrisdp_43",
        "mrisdp_44",
        "mrisdp_45",
        "mrisdp_46",
        "mrisdp_47",
        "mrisdp_48",
        "mrisdp_49",
        "mrisdp_50",
        "mrisdp_51",
        "mrisdp_52",
        "mrisdp_53",
        "mrisdp_54",
        "mrisdp_55",
        "mrisdp_56",
        "mrisdp_57",
        "mrisdp_58",
        "mrisdp_59",
        "mrisdp_60",
        "mrisdp_61",
        "mrisdp_62",
        "mrisdp_63",
        "mrisdp_64",
        "mrisdp_65",
        "mrisdp_66",
        "mrisdp_67",
        "mrisdp_68",
        "mrisdp_69",
        "mrisdp_70",
        "mrisdp_71",
        "mrisdp_72",
        "mrisdp_73",
        "mrisdp_74",
        "mrisdp_75",
        "mrisdp_76",
        "mrisdp_77",
        "mrisdp_78",
        "mrisdp_79",
        "mrisdp_80",
        "mrisdp_81",
        "mrisdp_82",
        "mrisdp_83",
        "mrisdp_84",
        "mrisdp_85",
        "mrisdp_86",
        "mrisdp_87",
        "mrisdp_88",
        "mrisdp_89",
        "mrisdp_90",
        "mrisdp_91",
        "mrisdp_92",
        "mrisdp_93",
        "mrisdp_94",
        "mrisdp_95",
        "mrisdp_96",
        "mrisdp_97",
        "mrisdp_98",
        "mrisdp_99",
        "mrisdp_100",
        "mrisdp_101",
        "mrisdp_102",
        "mrisdp_103",
        "mrisdp_104",
        "mrisdp_105",
        "mrisdp_106",
        "mrisdp_107",
        "mrisdp_108",
        "mrisdp_109",
        "mrisdp_110",
        "mrisdp_111",
        "mrisdp_112",
        "mrisdp_113",
        "mrisdp_114",
        "mrisdp_115",
        "mrisdp_116",
        "mrisdp_117",
        "mrisdp_118",
        "mrisdp_119",
        "mrisdp_120",
        "mrisdp_121",
        "mrisdp_122",
        "mrisdp_123",
        "mrisdp_124",
        "mrisdp_125",
        "mrisdp_126",
        "mrisdp_127",
        "mrisdp_128",
        "mrisdp_129",
        "mrisdp_130",
        "mrisdp_131",
        "mrisdp_132",
        "mrisdp_133",
        "mrisdp_134",
        "mrisdp_135",
        "mrisdp_136",
        "mrisdp_137",
        "mrisdp_138",
        "mrisdp_139",
        "mrisdp_140",
        "mrisdp_141",
        "mrisdp_142",
        "mrisdp_143",
        "mrisdp_144",
        "mrisdp_145",
        "mrisdp_146",
        "mrisdp_147",
        "mrisdp_148",
        "mrisdp_303",
        "mrisdp_304",
        "mrisdp_305",
        "mrisdp_306",
        "mrisdp_307",
        "mrisdp_308",
        "mrisdp_309",
        "mrisdp_310",
        "mrisdp_311",
        "mrisdp_312",
        "mrisdp_313",
        "mrisdp_314",
        "mrisdp_315",
        "mrisdp_316",
        "mrisdp_317",
        "mrisdp_318",
        "mrisdp_319",
        "mrisdp_320",
        "mrisdp_321",
        "mrisdp_322",
        "mrisdp_323",
        "mrisdp_324",
        "mrisdp_325",
        "mrisdp_326",
        "mrisdp_327",
        "mrisdp_328",
        "mrisdp_329",
        "mrisdp_330",
        "mrisdp_331",
        "mrisdp_332",
        "mrisdp_333",
        "mrisdp_334",
        "mrisdp_335",
        "mrisdp_336",
        "mrisdp_337",
        "mrisdp_338",
        "mrisdp_339",
        "mrisdp_340",
        "mrisdp_341",
        "mrisdp_342",
        "mrisdp_343",
        "mrisdp_344",
        "mrisdp_345",
        "mrisdp_346",
        "mrisdp_347",
        "mrisdp_348",
        "mrisdp_349",
        "mrisdp_350",
        "mrisdp_351",
        "mrisdp_352",
        "mrisdp_353",
        "mrisdp_354",
        "mrisdp_355",
        "mrisdp_356",
        "mrisdp_357",
        "mrisdp_358",
        "mrisdp_359",
        "mrisdp_360",
        "mrisdp_361",
        "mrisdp_362",
        "mrisdp_363",
        "mrisdp_364",
        "mrisdp_365",
        "mrisdp_366",
        "mrisdp_367",
        "mrisdp_368",
        "mrisdp_369",
        "mrisdp_370",
        "mrisdp_371",
        "mrisdp_372",
        "mrisdp_373",
        "mrisdp_374",
        "mrisdp_375",
        "mrisdp_376",
        "mrisdp_377",
        "mrisdp_378",
        "mrisdp_379",
        "mrisdp_380",
        "mrisdp_381",
        "mrisdp_382",
        "mrisdp_383",
        "mrisdp_384",
        "mrisdp_385",
        "mrisdp_386",
        "mrisdp_387",
        "mrisdp_388",
        "mrisdp_389",
        "mrisdp_390",
        "mrisdp_391",
        "mrisdp_392",
        "mrisdp_393",
        "mrisdp_394",
        "mrisdp_395",
        "mrisdp_396",
        "mrisdp_397",
        "mrisdp_398",
        "mrisdp_399",
        "mrisdp_400",
        "mrisdp_401",
        "mrisdp_402",
        "mrisdp_403",
        "mrisdp_404",
        "mrisdp_405",
        "mrisdp_406",
        "mrisdp_407",
        "mrisdp_408",
        "mrisdp_409",
        "mrisdp_410",
        "mrisdp_411",
        "mrisdp_412",
        "mrisdp_413",
        "mrisdp_414",
        "mrisdp_415",
        "mrisdp_416",
        "mrisdp_417",
        "mrisdp_418",
        "mrisdp_419",
        "mrisdp_420",
        "mrisdp_421",
        "mrisdp_422",
        "mrisdp_423",
        "mrisdp_424",
        "mrisdp_425",
        "mrisdp_426",
        "mrisdp_427",
        "mrisdp_428",
        "mrisdp_429",
        "mrisdp_430",
        "mrisdp_431",
        "mrisdp_432",
        "mrisdp_433",
        "mrisdp_434",
        "mrisdp_435",
        "mrisdp_436",
        "mrisdp_437",
        "mrisdp_438",
        "mrisdp_439",
        "mrisdp_440",
        "mrisdp_441",
        "mrisdp_442",
        "mrisdp_443",
        "mrisdp_444",
        "mrisdp_445",
        "mrisdp_446",
        "mrisdp_447",
        "mrisdp_448",
        "mrisdp_449",
        "mrisdp_450",
    ],
    "abcd_mrisdp20201": [
        "mrisdp_907",
        "mrisdp_908",
        "mrisdp_909",
        "mrisdp_910",
        "mrisdp_911",
        "mrisdp_912",
        "mrisdp_913",
        "mrisdp_914",
        "mrisdp_915",
        "mrisdp_916",
        "mrisdp_917",
        "mrisdp_918",
        "mrisdp_919",
        "mrisdp_920",
        "mrisdp_921",
        "mrisdp_922",
        "mrisdp_923",
        "mrisdp_924",
        "mrisdp_925",
        "mrisdp_926",
        "mrisdp_927",
        "mrisdp_928",
        "mrisdp_929",
        "mrisdp_930",
        "mrisdp_931",
        "mrisdp_932",
        "mrisdp_933",
        "mrisdp_934",
        "mrisdp_935",
        "mrisdp_936",
        "mrisdp_937",
        "mrisdp_938",
        "mrisdp_939",
        "mrisdp_940",
        "mrisdp_941",
        "mrisdp_942",
        "mrisdp_943",
        "mrisdp_944",
        "mrisdp_945",
        "mrisdp_946",
        "mrisdp_947",
        "mrisdp_948",
        "mrisdp_949",
        "mrisdp_950",
        "mrisdp_951",
        "mrisdp_952",
        "mrisdp_953",
        "mrisdp_954",
        "mrisdp_955",
        "mrisdp_956",
        "mrisdp_957",
        "mrisdp_958",
        "mrisdp_959",
        "mrisdp_960",
        "mrisdp_961",
        "mrisdp_962",
        "mrisdp_963",
        "mrisdp_964",
        "mrisdp_965",
        "mrisdp_966",
        "mrisdp_967",
        "mrisdp_968",
        "mrisdp_969",
        "mrisdp_970",
        "mrisdp_971",
        "mrisdp_972",
        "mrisdp_973",
        "mrisdp_974",
        "mrisdp_975",
        "mrisdp_976",
        "mrisdp_977",
        "mrisdp_978",
        "mrisdp_979",
        "mrisdp_980",
        "mrisdp_981",
        "mrisdp_982",
        "mrisdp_983",
        "mrisdp_984",
        "mrisdp_985",
        "mrisdp_986",
        "mrisdp_987",
        "mrisdp_988",
        "mrisdp_989",
        "mrisdp_990",
        "mrisdp_991",
        "mrisdp_992",
        "mrisdp_993",
        "mrisdp_994",
        "mrisdp_995",
        "mrisdp_996",
        "mrisdp_997",
        "mrisdp_998",
        "mrisdp_999",
        "mrisdp_1000",
        "mrisdp_1001",
        "mrisdp_1002",
        "mrisdp_1003",
        "mrisdp_1004",
        "mrisdp_1005",
        "mrisdp_1006",
        "mrisdp_1007",
        "mrisdp_1008",
        "mrisdp_1009",
        "mrisdp_1010",
        "mrisdp_1011",
        "mrisdp_1012",
        "mrisdp_1013",
        "mrisdp_1014",
        "mrisdp_1015",
        "mrisdp_1016",
        "mrisdp_1017",
        "mrisdp_1018",
        "mrisdp_1019",
        "mrisdp_1020",
        "mrisdp_1021",
        "mrisdp_1022",
        "mrisdp_1023",
        "mrisdp_1024",
        "mrisdp_1025",
        "mrisdp_1026",
        "mrisdp_1027",
        "mrisdp_1028",
        "mrisdp_1029",
        "mrisdp_1030",
        "mrisdp_1031",
        "mrisdp_1032",
        "mrisdp_1033",
        "mrisdp_1034",
        "mrisdp_1035",
        "mrisdp_1036",
        "mrisdp_1037",
        "mrisdp_1038",
        "mrisdp_1039",
        "mrisdp_1040",
        "mrisdp_1041",
        "mrisdp_1042",
        "mrisdp_1043",
        "mrisdp_1044",
        "mrisdp_1045",
        "mrisdp_1046",
        "mrisdp_1047",
        "mrisdp_1048",
        "mrisdp_1049",
        "mrisdp_1050",
        "mrisdp_1051",
        "mrisdp_1052",
        "mrisdp_1053",
        "mrisdp_1054",
    ],
    "abcd_dti_p101": [
        "dmri_dtifa_fiberat_fxrh",
        "dmri_dtifa_fiberat_fxlh",
        "dmri_dtifa_fiberat_cgcrh",
        "dmri_dtifa_fiberat_cgclh",
        "dmri_dtifa_fiberat_cghrh",
        "dmri_dtifa_fiberat_cghlh",
        "dmri_dtifa_fiberat_cstrh",
        "dmri_dtifa_fiberat_cstlh",
        "dmri_dtifa_fiberat_atrrh",
        "dmri_dtifa_fiberat_atrlh",
        "dmri_dtifa_fiberat_uncrh",
        "dmri_dtifa_fiberat_unclh",
        "dmri_dtifa_fiberat_ilfrh",
        "dmri_dtifa_fiberat_ilflh",
        "dmri_dtifa_fiberat_iforh",
        "dmri_dtifa_fiberat_ifolh",
        "dmri_dtifa_fiberat_fmaj",
        "dmri_dtifa_fiberat_fmin",
        "dmri_dtifa_fiberat_cc",
        "dmri_dtifa_fiberat_slfrh",
        "dmri_dtifa_fiberat_slflh",
        "dmri_dtifa_fiberat_tslfrh",
        "dmri_dtifa_fiberat_tslflh",
        "dmri_dtifa_fiberat_pslfrh",
        "dmri_dtifa_fiberat_pslflh",
        "dmri_dtifa_fiberat_scsrh",
        "dmri_dtifa_fiberat_scslh",
        "dmri_dtifa_fiberat_fscsrh",
        "dmri_dtifa_fiberat_fscslh",
        "dmri_dtifa_fiberat_pscsrh",
        "dmri_dtifa_fiberat_pscslh",
        "dmri_dtifa_fiberat_sifcrh",
        "dmri_dtifa_fiberat_sifclh",
        "dmri_dtifa_fiberat_ifsfcrh",
        "dmri_dtifa_fiberat_ifsfclh",
        "dmri_dtifa_fiberat_fxcutrh",
        "dmri_dtifa_fiberat_fxcutlh",
        "dmri_dtimd_fiberat_fxrh",
        "dmri_dtimd_fiberat_fxlh",
        "dmri_dtimd_fiberat_cgcrh",
        "dmri_dtimd_fiberat_cgclh",
        "dmri_dtimd_fiberat_cghrh",
        "dmri_dtimd_fiberat_cghlh",
        "dmri_dtimd_fiberat_cstrh",
        "dmri_dtimd_fiberat_cstlh",
        "dmri_dtimd_fiberat_atrrh",
        "dmri_dtimd_fiberat_atrlh",
        "dmri_dtimd_fiberat_uncrh",
        "dmri_dtimd_fiberat_unclh",
        "dmri_dtimd_fiberat_ilfrh",
        "dmri_dtimd_fiberat_ilflh",
        "dmri_dtimd_fiberat_iforh",
        "dmri_dtimd_fiberat_ifolh",
        "dmri_dtimd_fiberat_fmaj",
        "dmri_dtimd_fiberat_fmin",
        "dmri_dtimd_fiberat_cc",
        "dmri_dtimd_fiberat_slfrh",
        "dmri_dtimd_fiberat_slflh",
        "dmri_dtimd_fiberat_tslfrh",
        "dmri_dtimd_fiberat_tslflh",
        "dmri_dtimd_fiberat_pslfrh",
        "dmri_dtimd_fiberat_pslflh",
        "dmri_dtimd_fiberat_scsrh",
        "dmri_dtimd_fiberat_scslh",
        "dmri_dtimd_fiberat_fscsrh",
        "dmri_dtimd_fiberat_fscslh",
        "dmri_dtimd_fiberat_pscsrh",
        "dmri_dtimd_fiberat_pscslh",
        "dmri_dtimd_fiberat_sifcrh",
        "dmri_dtimd_fiberat_sifclh",
        "dmri_dtimd_fiberat_ifsfcrh",
        "dmri_dtimd_fiberat_ifsfclh",
        "dmri_dtimd_fiberat_fxcutrh",
        "dmri_dtimd_fiberat_fxcutlh",
        "dmri_dtivol_fiberat_fxrh",
        "dmri_dtivol_fiberat_fxlh",
        "dmri_dtivol_fiberat_cgcrh",
        "dmri_dtivol_fiberat_cgclh",
        "dmri_dtivol_fiberat_cghrh",
        "dmri_dtivol_fiberat_cghlh",
        "dmri_dtivol_fiberat_cstrh",
        "dmri_dtivol_fiberat_cstlh",
        "dmri_dtivol_fiberat_atrrh",
        "dmri_dtivol_fiberat_atrlh",
        "dmri_dtivol_fiberat_uncrh",
        "dmri_dtivol_fiberat_unclh",
        "dmri_dtivol_fiberat_ilfrh",
        "dmri_dtivol_fiberat_ilflh",
        "dmri_dtivol_fiberat_iforh",
        "dmri_dtivol_fiberat_ifolh",
        "dmri_dtivol_fiberat_fmaj",
        "dmri_dtivol_fiberat_fmin",
        "dmri_dtivol_fiberat_cc",
        "dmri_dtivol_fiberat_slfrh",
        "dmri_dtivol_fiberat_slflh",
        "dmri_dtivol_fiberat_tslfrh",
        "dmri_dtivol_fiberat_tslflh",
        "dmri_dtivol_fiberat_pslfrh",
        "dmri_dtivol_fiberat_pslflh",
        "dmri_dtivol_fiberat_scsrh",
        "dmri_dtivol_fiberat_scslh",
        "dmri_dtivol_fiberat_fscsrh",
        "dmri_dtivol_fiberat_fscslh",
        "dmri_dtivol_fiberat_pscsrh",
        "dmri_dtivol_fiberat_pscslh",
        "dmri_dtivol_fiberat_sifcrh",
        "dmri_dtivol_fiberat_sifclh",
        "dmri_dtivol_fiberat_ifsfcrh",
        "dmri_dtivol_fiberat_ifsfclh",
        "dmri_dtivol_fiberat_fxcutrh",
        "dmri_dtivol_fiberat_fxcutlh",
    ],
    "abcd_drsip101": [
        "dmri_rsirni_fib_fxrh",
        "dmri_rsirni_fib_fxlh",
        "dmri_rsirni_fib_cgcrh",
        "dmri_rsirni_fib_cgclh",
        "dmri_rsirni_fib_cghrh",
        "dmri_rsirni_fib_cghlh",
        "dmri_rsirni_fib_cstrh",
        "dmri_rsirni_fib_cstlh",
        "dmri_rsirni_fib_atrrh",
        "dmri_rsirni_fib_atrlh",
        "dmri_rsirni_fib_uncrh",
        "dmri_rsirni_fib_unclh",
        "dmri_rsirni_fib_ilfrh",
        "dmri_rsirni_fib_ilflh",
        "dmri_rsirni_fib_iforh",
        "dmri_rsirni_fib_ifolh",
        "dmri_rsirni_fib_fmaj",
        "dmri_rsirni_fib_fmin",
        "dmri_rsirni_fib_cc",
        "dmri_rsirni_fib_slfrh",
        "dmri_rsirni_fib_slflh",
        "dmri_rsirni_fib_tslfrh",
        "dmri_rsirni_fib_tslflh",
        "dmri_rsirni_fib_pslfrh",
        "dmri_rsirni_fib_pslflh",
        "dmri_rsirni_fib_scsrh",
        "dmri_rsirni_fib_scslh",
        "dmri_rsirni_fib_fscsrh",
        "dmri_rsirni_fib_fscslh",
        "dmri_rsirni_fib_pscsrh",
        "dmri_rsirni_fib_pscslh",
        "dmri_rsirni_fib_sifcrh",
        "dmri_rsirni_fib_sifclh",
        "dmri_rsirni_fib_ifsfcrh",
        "dmri_rsirni_fib_ifsfclh",
        "dmri_rsirni_fib_fxcutrh",
        "dmri_rsirni_fib_fxcutlh",
        "dmri_rsirni_scs_ilvlh",
        "dmri_rsirni_scs_cbwmlh",
        "dmri_rsirni_scs_cbclh",
        "dmri_rsirni_scs_tplh",
        "dmri_rsirni_scs_cdlh",
        "dmri_rsirni_scs_ptlh",
        "dmri_rsirni_scs_pllh",
        "dmri_rsirni_scs_bs",
        "dmri_rsirni_scs_hclh",
        "dmri_rsirni_scs_aglh",
        "dmri_rsirni_scs_ablh",
        "dmri_rsirni_scs_vdclh",
        "dmri_rsirni_scs_cwmrh",
        "dmri_rsirni_scs_lvrh",
        "dmri_rsirni_scs_ilvrh",
        "dmri_rsirni_scs_cbwmrh",
        "dmri_rsirni_scs_cbcrh",
        "dmri_rsirni_scs_tprh",
        "dmri_rsirni_scs_cdrh",
        "dmri_rsirni_scs_ptrh",
        "dmri_rsirni_scs_plrh",
        "dmri_rsirni_scs_hcrh",
        "dmri_rsirni_scs_agrh",
        "dmri_rsirni_scs_abrh",
        "dmri_rsirni_scs_vdcrh",
    ],
    "abcd_drsip201": [
        "dmri_rsirnd_fib_fxrh",
        "dmri_rsirnd_fib_fxlh",
        "dmri_rsirnd_fib_cgcrh",
        "dmri_rsirnd_fib_cgclh",
        "dmri_rsirnd_fib_cghrh",
        "dmri_rsirnd_fib_cghlh",
        "dmri_rsirnd_fib_cstrh",
        "dmri_rsirnd_fib_cstlh",
        "dmri_rsirnd_fib_atrrh",
        "dmri_rsirnd_fib_atrlh",
        "dmri_rsirnd_fib_uncrh",
        "dmri_rsirnd_fib_unclh",
        "dmri_rsirnd_fib_ilfrh",
        "dmri_rsirnd_fib_ilflh",
        "dmri_rsirnd_fib_iforh",
        "dmri_rsirnd_fib_ifolh",
        "dmri_rsirnd_fib_fmaj",
        "dmri_rsirnd_fib_fmin",
        "dmri_rsirnd_fib_cc",
        "dmri_rsirnd_fib_slfrh",
        "dmri_rsirnd_fib_slflh",
        "dmri_rsirnd_fib_tslfrh",
        "dmri_rsirnd_fib_tslflh",
        "dmri_rsirnd_fib_pslfrh",
        "dmri_rsirnd_fib_pslflh",
        "dmri_rsirnd_fib_scsrh",
        "dmri_rsirnd_fib_scslh",
        "dmri_rsirnd_fib_fscsrh",
        "dmri_rsirnd_fib_fscslh",
        "dmri_rsirnd_fib_pscsrh",
        "dmri_rsirnd_fib_pscslh",
        "dmri_rsirnd_fib_sifcrh",
        "dmri_rsirnd_fib_sifclh",
        "dmri_rsirnd_fib_ifsfcrh",
        "dmri_rsirnd_fib_ifsfclh",
        "dmri_rsirnd_fib_fxcutrh",
        "dmri_rsirnd_fib_fxcutlh",
        "dmri_rsirnd_scs_cwmlh",
        "dmri_rsirnd_scs_cbwmlh",
        "dmri_rsirnd_scs_cbclh",
        "dmri_rsirnd_scs_tplh",
        "dmri_rsirnd_scs_cdlh",
        "dmri_rsirnd_scs_ptlh",
        "dmri_rsirnd_scs_pllh",
        "dmri_rsirnd_scs_bs",
        "dmri_rsirnd_scs_hclh",
        "dmri_rsirnd_scs_aglh",
        "dmri_rsirnd_scs_ablh",
        "dmri_rsirnd_scs_vdclh",
        "dmri_rsirnd_scs_cwmrh",
        "dmri_rsirnd_scs_cbwmrh",
        "dmri_rsirnd_scs_cbcrh",
        "dmri_rsirnd_scs_tprh",
        "dmri_rsirnd_scs_cdrh",
        "dmri_rsirnd_scs_ptrh",
        "dmri_rsirnd_scs_plrh",
        "dmri_rsirnd_scs_hcrh",
        "dmri_rsirnd_scs_agrh",
        "dmri_rsirnd_scs_abrh",
        "dmri_rsirnd_scs_vdcrh",
    ],
    "abcd_mrirsfd01": [
        "mrirsfd1",
        "mrirsfd2",
        "mrirsfd3",
        "mrirsfd4",
        "mrirsfd5",
        "mrirsfd6",
        "mrirsfd7",
        "mrirsfd8",
        "mrirsfd9",
        "mrirsfd10",
        "mrirsfd11",
        "mrirsfd12",
        "mrirsfd13",
        "mrirsfd14",
        "mrirsfd15",
        "mrirsfd16",
        "mrirsfd17",
        "mrirsfd18",
        "mrirsfd19",
        "mrirsfd20",
        "mrirsfd21",
        "mrirsfd22",
        "mrirsfd23",
        "mrirsfd24",
        "mrirsfd25",
        "mrirsfd26",
        "mrirsfd27",
        "mrirsfd28",
        "mrirsfd29",
        "mrirsfd30",
        "mrirsfd31",
        "mrirsfd32",
        "mrirsfd33",
        "mrirsfd34",
        "mrirsfd35",
        "mrirsfd36",
        "mrirsfd37",
        "mrirsfd38",
        "mrirsfd39",
        "mrirsfd40",
        "mrirsfd41",
        "mrirsfd42",
        "mrirsfd43",
        "mrirsfd44",
        "mrirsfd45",
        "mrirsfd46",
        "mrirsfd47",
        "mrirsfd48",
        "mrirsfd49",
        "mrirsfd50",
        "mrirsfd51",
        "mrirsfd52",
        "mrirsfd53",
        "mrirsfd54",
        "mrirsfd55",
        "mrirsfd56",
        "mrirsfd57",
        "mrirsfd58",
        "mrirsfd59",
        "mrirsfd60",
        "mrirsfd61",
        "mrirsfd62",
        "mrirsfd63",
        "mrirsfd64",
        "mrirsfd65",
        "mrirsfd66",
        "mrirsfd67",
        "mrirsfd68",
        "mrirsfd69",
        "mrirsfd70",
        "mrirsfd71",
        "mrirsfd72",
        "mrirsfd73",
        "mrirsfd74",
        "mrirsfd75",
        "mrirsfd76",
        "mrirsfd77",
        "mrirsfd78",
        "mrirsfd79",
        "mrirsfd80",
        "mrirsfd81",
        "mrirsfd82",
        "mrirsfd83",
        "mrirsfd84",
        "mrirsfd85",
        "mrirsfd86",
        "mrirsfd87",
        "mrirsfd88",
        "mrirsfd89",
        "mrirsfd90",
        "mrirsfd91",
        "mrirsfd92",
        "mrirsfd93",
        "mrirsfd94",
        "mrirsfd95",
        "mrirsfd96",
        "mrirsfd97",
        "mrirsfd98",
        "mrirsfd99",
        "mrirsfd100",
        "mrirsfd101",
        "mrirsfd102",
        "mrirsfd103",
        "mrirsfd104",
        "mrirsfd105",
        "mrirsfd106",
        "mrirsfd107",
        "mrirsfd108",
        "mrirsfd109",
        "mrirsfd110",
        "mrirsfd111",
        "mrirsfd112",
        "mrirsfd113",
        "mrirsfd114",
        "mrirsfd115",
        "mrirsfd116",
        "mrirsfd117",
        "mrirsfd118",
        "mrirsfd119",
        "mrirsfd120",
        "mrirsfd121",
        "mrirsfd122",
        "mrirsfd123",
        "mrirsfd124",
        "mrirsfd125",
        "mrirsfd126",
        "mrirsfd127",
        "mrirsfd128",
        "mrirsfd129",
        "mrirsfd130",
        "mrirsfd131",
        "mrirsfd132",
        "mrirsfd133",
        "mrirsfd134",
        "mrirsfd135",
        "mrirsfd136",
        "mrirsfd137",
        "mrirsfd138",
        "mrirsfd139",
        "mrirsfd140",
        "mrirsfd141",
        "mrirsfd142",
        "mrirsfd143",
        "mrirsfd144",
        "mrirsfd145",
        "mrirsfd146",
        "mrirsfd147",
        "mrirsfd148",
    ],
    "abcd_mrirstv02": [
        "rsfmri_c_ngd_meanmotion",
        "rsfmri_c_ngd_ntpoints",
        "rsfmri_var_cdk_banksstslh",
        "rsfmri_var_cdk_cdaclatelh",
        "rsfmri_var_cdk_cdmdflh",
        "rsfmri_var_cdk_cuneuslh",
        "rsfmri_var_cdk_entorhinallh",
        "rsfmri_var_cdk_fflh",
        "rsfmri_var_cdk_ifpalh",
        "rsfmri_var_cdk_iftlh",
        "rsfmri_var_cdk_ihclatelh",
        "rsfmri_var_cdk_loccipitallh",
        "rsfmri_var_cdk_loboflh",
        "rsfmri_var_cdk_linguallh",
        "rsfmri_var_cdk_moboflh",
        "rsfmri_var_cdk_mdtlh",
        "rsfmri_var_cdk_parahpallh",
        "rsfmri_var_cdk_paracentrallh",
        "rsfmri_var_cdk_parsopllh",
        "rsfmri_var_cdk_parsobalislh",
        "rsfmri_var_cdk_parstularislh",
        "rsfmri_var_cdk_pericclh",
        "rsfmri_var_cdk_postcentrallh",
        "rsfmri_var_cdk_psclatelh",
        "rsfmri_var_cdk_precentrallh",
        "rsfmri_var_cdk_precuneuslh",
        "rsfmri_var_cdk_rlaclatelh",
        "rsfmri_var_cdk_rlmdflh",
        "rsfmri_var_cdk_suflh",
        "rsfmri_var_cdk_spetallh",
        "rsfmri_var_cdk_sutlh",
        "rsfmri_var_cdk_smlh",
        "rsfmri_var_cdk_fpolelh",
        "rsfmri_var_cdk_tpolelh",
        "rsfmri_var_cdk_tvtlh",
        "rsfmri_var_cdk_insulalh",
        "rsfmri_var_cdk_banksstsrh",
        "rsfmri_var_cdk_cdaclaterh",
        "rsfmri_var_cdk_cdmdfrh",
        "rsfmri_var_cdk_cuneusrh",
        "rsfmri_var_cdk_entorhinalrh",
        "rsfmri_var_cdk_ffrh",
        "rsfmri_var_cdk_ifparh",
        "rsfmri_var_cdk_iftrh",
        "rsfmri_var_cdk_ihclaterh",
        "rsfmri_var_cdk_loccipitalrh",
        "rsfmri_var_cdk_lobofrh",
        "rsfmri_var_cdk_lingualrh",
        "rsfmri_var_cdk_mobofrh",
        "rsfmri_var_cdk_mdtrh",
        "rsfmri_var_cdk_parahpalrh",
        "rsfmri_var_cdk_paracentralrh",
        "rsfmri_var_cdk_parsoplrh",
        "rsfmri_var_cdk_parsobalisrh",
        "rsfmri_var_cdk_parstularisrh",
        "rsfmri_var_cdk_periccrh",
        "rsfmri_var_cdk_postcentralrh",
        "rsfmri_var_cdk_psclaterh",
        "rsfmri_var_cdk_precentralrh",
        "rsfmri_var_cdk_precuneusrh",
        "rsfmri_var_cdk_rlaclaterh",
        "rsfmri_var_cdk_rlmdfrh",
        "rsfmri_var_cdk_sufrh",
        "rsfmri_var_cdk_spetalrh",
        "rsfmri_var_cdk_sutrh",
        "rsfmri_var_cdk_smrh",
        "rsfmri_var_cdk_fpolerh",
        "rsfmri_var_cdk_tpolerh",
        "rsfmri_var_cdk_tvtrh",
        "rsfmri_var_cdk_insularh",
        "rsfmri_var_cortgordon_gp1lh",
        "rsfmri_var_cortgordon_gp2lh",
        "rsfmri_var_cortgordon_gp3lh",
        "rsfmri_var_cortgordon_gp4lh",
        "rsfmri_var_cortgordon_gp5lh",
        "rsfmri_var_cortgordon_gp6lh",
        "rsfmri_var_cortgordon_gp7lh",
        "rsfmri_var_cortgordon_gp8lh",
        "rsfmri_var_cortgordon_gp9lh",
        "rsfmri_var_cortgordon_gp10lh",
        "rsfmri_var_cortgordon_gp11lh",
        "rsfmri_var_cortgordon_gp12lh",
        "rsfmri_var_cortgordon_gp13lh",
        "rsfmri_var_cortgordon_gp14lh",
        "rsfmri_var_cortgordon_gp15lh",
        "rsfmri_var_cortgordon_gp16lh",
        "rsfmri_var_cortgordon_gp17lh",
        "rsfmri_var_cortgordon_gp18lh",
        "rsfmri_var_cortgordon_gp19lh",
        "rsfmri_var_cortgordon_gp20lh",
        "rsfmri_var_cortgordon_gp21lh",
        "rsfmri_var_cortgordon_gp22lh",
        "rsfmri_var_cortgordon_gp23lh",
        "rsfmri_var_cortgordon_gp24lh",
        "rsfmri_var_cortgordon_gp25lh",
        "rsfmri_var_cortgordon_gp26lh",
        "rsfmri_var_cortgordon_gp27lh",
        "rsfmri_var_cortgordon_gp28lh",
        "rsfmri_var_cortgordon_gp29lh",
        "rsfmri_var_cortgordon_gp30lh",
        "rsfmri_var_cortgordon_gp31lh",
        "rsfmri_var_cortgordon_gp32lh",
        "rsfmri_var_cortgordon_gp33lh",
        "rsfmri_var_cortgordon_gp34lh",
        "rsfmri_var_cortgordon_gp35lh",
        "rsfmri_var_cortgordon_gp36lh",
        "rsfmri_var_cortgordon_gp37lh",
        "rsfmri_var_cortgordon_gp38lh",
        "rsfmri_var_cortgordon_gp39lh",
        "rsfmri_var_cortgordon_gp40lh",
        "rsfmri_var_cortgordon_gp41lh",
        "rsfmri_var_cortgordon_gp42lh",
        "rsfmri_var_cortgordon_gp43lh",
        "rsfmri_var_cortgordon_gp44lh",
        "rsfmri_var_cortgordon_gp45lh",
        "rsfmri_var_cortgordon_gp46lh",
        "rsfmri_var_cortgordon_gp47lh",
        "rsfmri_var_cortgordon_gp48lh",
        "rsfmri_var_cortgordon_gp49lh",
        "rsfmri_var_cortgordon_gp50lh",
        "rsfmri_var_cortgordon_gp51lh",
        "rsfmri_var_cortgordon_gp52lh",
        "rsfmri_var_cortgordon_gp53lh",
        "rsfmri_var_cortgordon_gp54lh",
        "rsfmri_var_cortgordon_gp55lh",
        "rsfmri_var_cortgordon_gp56lh",
        "rsfmri_var_cortgordon_gp57lh",
        "rsfmri_var_cortgordon_gp58lh",
        "rsfmri_var_cortgordon_gp59lh",
        "rsfmri_var_cortgordon_gp60lh",
        "rsfmri_var_cortgordon_gp61lh",
        "rsfmri_var_cortgordon_gp62lh",
        "rsfmri_var_cortgordon_gp63lh",
        "rsfmri_var_cortgordon_gp64lh",
        "rsfmri_var_cortgordon_gp65lh",
        "rsfmri_var_cortgordon_gp66lh",
        "rsfmri_var_cortgordon_gp67lh",
        "rsfmri_var_cortgordon_gp68lh",
        "rsfmri_var_cortgordon_gp69lh",
        "rsfmri_var_cortgordon_gp70lh",
        "rsfmri_var_cortgordon_gp71lh",
        "rsfmri_var_cortgordon_gp72lh",
        "rsfmri_var_cortgordon_gp73lh",
        "rsfmri_var_cortgordon_gp74lh",
        "rsfmri_var_cortgordon_gp75lh",
        "rsfmri_var_cortgordon_gp76lh",
        "rsfmri_var_cortgordon_gp77lh",
        "rsfmri_var_cortgordon_gp78lh",
        "rsfmri_var_cortgordon_gp79lh",
        "rsfmri_var_cortgordon_gp80lh",
        "rsfmri_var_cortgordon_gp81lh",
        "rsfmri_var_cortgordon_gp82lh",
        "rsfmri_var_cortgordon_gp83lh",
        "rsfmri_var_cortgordon_gp84lh",
        "rsfmri_var_cortgordon_gp85lh",
        "rsfmri_var_cortgordon_gp86lh",
        "rsfmri_var_cortgordon_gp87lh",
        "rsfmri_var_cortgordon_gp88lh",
        "rsfmri_var_cortgordon_gp89lh",
        "rsfmri_var_cortgordon_gp90lh",
        "rsfmri_var_cortgordon_gp91lh",
        "rsfmri_var_cortgordon_gp92lh",
        "rsfmri_var_cortgordon_gp93lh",
        "rsfmri_var_cortgordon_gp94lh",
        "rsfmri_var_cortgordon_gp95lh",
        "rsfmri_var_cortgordon_gp96lh",
        "rsfmri_var_cortgordon_gp97lh",
        "rsfmri_var_cortgordon_gp98lh",
        "rsfmri_var_cortgordon_gp99lh",
        "rsfmri_var_cortgordon_gp100lh",
        "rsfmri_var_cortgordon_gp101lh",
        "rsfmri_var_cortgordon_gp102lh",
        "rsfmri_var_cortgordon_gp103lh",
        "rsfmri_var_cortgordon_gp104lh",
        "rsfmri_var_cortgordon_gp105lh",
        "rsfmri_var_cortgordon_gp106lh",
        "rsfmri_var_cortgordon_gp107lh",
        "rsfmri_var_cortgordon_gp108lh",
        "rsfmri_var_cortgordon_gp109lh",
        "rsfmri_var_cortgordon_gp110lh",
        "rsfmri_var_cortgordon_gp111lh",
        "rsfmri_var_cortgordon_gp112lh",
        "rsfmri_var_cortgordon_gp113lh",
        "rsfmri_var_cortgordon_gp114lh",
        "rsfmri_var_cortgordon_gp115lh",
        "rsfmri_var_cortgordon_gp116lh",
        "rsfmri_var_cortgordon_gp117lh",
        "rsfmri_var_cortgordon_gp118lh",
        "rsfmri_var_cortgordon_gp119lh",
        "rsfmri_var_cortgordon_gp120lh",
        "rsfmri_var_cortgordon_gp121lh",
        "rsfmri_var_cortgordon_gp122lh",
        "rsfmri_var_cortgordon_gp123lh",
        "rsfmri_var_cortgordon_gp124lh",
        "rsfmri_var_cortgordon_gp125lh",
        "rsfmri_var_cortgordon_gp126lh",
        "rsfmri_var_cortgordon_gp127lh",
        "rsfmri_var_cortgordon_gp128lh",
        "rsfmri_var_cortgordon_gp129lh",
        "rsfmri_var_cortgordon_gp130lh",
        "rsfmri_var_cortgordon_gp131lh",
        "rsfmri_var_cortgordon_gp132lh",
        "rsfmri_var_cortgordon_gp133lh",
        "rsfmri_var_cortgordon_gp134lh",
        "rsfmri_var_cortgordon_gp135lh",
        "rsfmri_var_cortgordon_gp136lh",
        "rsfmri_var_cortgordon_gp137lh",
        "rsfmri_var_cortgordon_gp138lh",
        "rsfmri_var_cortgordon_gp139lh",
        "rsfmri_var_cortgordon_gp140lh",
        "rsfmri_var_cortgordon_gp141lh",
        "rsfmri_var_cortgordon_gp142lh",
        "rsfmri_var_cortgordon_gp143lh",
        "rsfmri_var_cortgordon_gp144lh",
        "rsfmri_var_cortgordon_gp145lh",
        "rsfmri_var_cortgordon_gp146lh",
        "rsfmri_var_cortgordon_gp147lh",
        "rsfmri_var_cortgordon_gp148lh",
        "rsfmri_var_cortgordon_gp149lh",
        "rsfmri_var_cortgordon_gp150lh",
        "rsfmri_var_cortgordon_gp151lh",
        "rsfmri_var_cortgordon_gp152lh",
        "rsfmri_var_cortgordon_gp153lh",
        "rsfmri_var_cortgordon_gp154lh",
        "rsfmri_var_cortgordon_gp155lh",
        "rsfmri_var_cortgordon_gp156lh",
        "rsfmri_var_cortgordon_gp157lh",
        "rsfmri_var_cortgordon_gp158lh",
        "rsfmri_var_cortgordon_gp159lh",
        "rsfmri_var_cortgordon_gp160lh",
        "rsfmri_var_cortgordon_gp161lh",
        "rsfmri_var_cortgordon_gp162rh",
        "rsfmri_var_cortgordon_gp163rh",
        "rsfmri_var_cortgordon_gp164rh",
        "rsfmri_var_cortgordon_gp165rh",
        "rsfmri_var_cortgordon_gp166rh",
        "rsfmri_var_cortgordon_gp167rh",
        "rsfmri_var_cortgordon_gp168rh",
        "rsfmri_var_cortgordon_gp169rh",
        "rsfmri_var_cortgordon_gp170rh",
        "rsfmri_var_cortgordon_gp171rh",
        "rsfmri_var_cortgordon_gp172rh",
        "rsfmri_var_cortgordon_gp173rh",
        "rsfmri_var_cortgordon_gp174rh",
        "rsfmri_var_cortgordon_gp175rh",
        "rsfmri_var_cortgordon_gp176rh",
        "rsfmri_var_cortgordon_gp177rh",
        "rsfmri_var_cortgordon_gp178rh",
        "rsfmri_var_cortgordon_gp179rh",
        "rsfmri_var_cortgordon_gp180rh",
        "rsfmri_var_cortgordon_gp181rh",
        "rsfmri_var_cortgordon_gp182rh",
        "rsfmri_var_cortgordon_gp183rh",
        "rsfmri_var_cortgordon_gp184rh",
        "rsfmri_var_cortgordon_gp185rh",
        "rsfmri_var_cortgordon_gp186rh",
        "rsfmri_var_cortgordon_gp187rh",
        "rsfmri_var_cortgordon_gp188rh",
        "rsfmri_var_cortgordon_gp189rh",
        "rsfmri_var_cortgordon_gp190rh",
        "rsfmri_var_cortgordon_gp191rh",
        "rsfmri_var_cortgordon_gp192rh",
        "rsfmri_var_cortgordon_gp193rh",
        "rsfmri_var_cortgordon_gp194rh",
        "rsfmri_var_cortgordon_gp195rh",
        "rsfmri_var_cortgordon_gp196rh",
        "rsfmri_var_cortgordon_gp197rh",
        "rsfmri_var_cortgordon_gp198rh",
        "rsfmri_var_cortgordon_gp199rh",
        "rsfmri_var_cortgordon_gp200rh",
        "rsfmri_var_cortgordon_gp201rh",
        "rsfmri_var_cortgordon_gp202rh",
        "rsfmri_var_cortgordon_gp203rh",
        "rsfmri_var_cortgordon_gp204rh",
        "rsfmri_var_cortgordon_gp205rh",
        "rsfmri_var_cortgordon_gp206rh",
        "rsfmri_var_cortgordon_gp207rh",
        "rsfmri_var_cortgordon_gp208rh",
        "rsfmri_var_cortgordon_gp209rh",
        "rsfmri_var_cortgordon_gp210rh",
        "rsfmri_var_cortgordon_gp211rh",
        "rsfmri_var_cortgordon_gp212rh",
        "rsfmri_var_cortgordon_gp213rh",
        "rsfmri_var_cortgordon_gp214rh",
        "rsfmri_var_cortgordon_gp215rh",
        "rsfmri_var_cortgordon_gp216rh",
        "rsfmri_var_cortgordon_gp217rh",
        "rsfmri_var_cortgordon_gp218rh",
        "rsfmri_var_cortgordon_gp219rh",
        "rsfmri_var_cortgordon_gp220rh",
        "rsfmri_var_cortgordon_gp221rh",
        "rsfmri_var_cortgordon_gp222rh",
        "rsfmri_var_cortgordon_gp223rh",
        "rsfmri_var_cortgordon_gp224rh",
        "rsfmri_var_cortgordon_gp225rh",
        "rsfmri_var_cortgordon_gp226rh",
        "rsfmri_var_cortgordon_gp227rh",
        "rsfmri_var_cortgordon_gp228rh",
        "rsfmri_var_cortgordon_gp229rh",
        "rsfmri_var_cortgordon_gp230rh",
        "rsfmri_var_cortgordon_gp231rh",
        "rsfmri_var_cortgordon_gp232rh",
        "rsfmri_var_cortgordon_gp233rh",
        "rsfmri_var_cortgordon_gp234rh",
        "rsfmri_var_cortgordon_gp235rh",
        "rsfmri_var_cortgordon_gp236rh",
        "rsfmri_var_cortgordon_gp237rh",
        "rsfmri_var_cortgordon_gp238rh",
        "rsfmri_var_cortgordon_gp239rh",
        "rsfmri_var_cortgordon_gp240rh",
        "rsfmri_var_cortgordon_gp241rh",
        "rsfmri_var_cortgordon_gp242rh",
        "rsfmri_var_cortgordon_gp243rh",
        "rsfmri_var_cortgordon_gp244rh",
        "rsfmri_var_cortgordon_gp245rh",
        "rsfmri_var_cortgordon_gp246rh",
        "rsfmri_var_cortgordon_gp247rh",
        "rsfmri_var_cortgordon_gp248rh",
        "rsfmri_var_cortgordon_gp249rh",
        "rsfmri_var_cortgordon_gp250rh",
        "rsfmri_var_cortgordon_gp251rh",
        "rsfmri_var_cortgordon_gp252rh",
        "rsfmri_var_cortgordon_gp253rh",
        "rsfmri_var_cortgordon_gp254rh",
        "rsfmri_var_cortgordon_gp255rh",
        "rsfmri_var_cortgordon_gp256rh",
        "rsfmri_var_cortgordon_gp257rh",
        "rsfmri_var_cortgordon_gp258rh",
        "rsfmri_var_cortgordon_gp259rh",
        "rsfmri_var_cortgordon_gp260rh",
        "rsfmri_var_cortgordon_gp261rh",
        "rsfmri_var_cortgordon_gp262rh",
        "rsfmri_var_cortgordon_gp263rh",
        "rsfmri_var_cortgordon_gp264rh",
        "rsfmri_var_cortgordon_gp265rh",
        "rsfmri_var_cortgordon_gp266rh",
        "rsfmri_var_cortgordon_gp267rh",
        "rsfmri_var_cortgordon_gp268rh",
        "rsfmri_var_cortgordon_gp269rh",
        "rsfmri_var_cortgordon_gp270rh",
        "rsfmri_var_cortgordon_gp271rh",
        "rsfmri_var_cortgordon_gp272rh",
        "rsfmri_var_cortgordon_gp273rh",
        "rsfmri_var_cortgordon_gp274rh",
        "rsfmri_var_cortgordon_gp275rh",
        "rsfmri_var_cortgordon_gp276rh",
        "rsfmri_var_cortgordon_gp277rh",
        "rsfmri_var_cortgordon_gp278rh",
        "rsfmri_var_cortgordon_gp279rh",
        "rsfmri_var_cortgordon_gp280rh",
        "rsfmri_var_cortgordon_gp281rh",
        "rsfmri_var_cortgordon_gp282rh",
        "rsfmri_var_cortgordon_gp283rh",
        "rsfmri_var_cortgordon_gp284rh",
        "rsfmri_var_cortgordon_gp285rh",
        "rsfmri_var_cortgordon_gp286rh",
        "rsfmri_var_cortgordon_gp287rh",
        "rsfmri_var_cortgordon_gp288rh",
        "rsfmri_var_cortgordon_gp289rh",
        "rsfmri_var_cortgordon_gp290rh",
        "rsfmri_var_cortgordon_gp291rh",
        "rsfmri_var_cortgordon_gp292rh",
        "rsfmri_var_cortgordon_gp293rh",
        "rsfmri_var_cortgordon_gp294rh",
        "rsfmri_var_cortgordon_gp295rh",
        "rsfmri_var_cortgordon_gp296rh",
        "rsfmri_var_cortgordon_gp297rh",
        "rsfmri_var_cortgordon_gp298rh",
        "rsfmri_var_cortgordon_gp299rh",
        "rsfmri_var_cortgordon_gp300rh",
        "rsfmri_var_cortgordon_gp301rh",
        "rsfmri_var_cortgordon_gp302rh",
        "rsfmri_var_cortgordon_gp303rh",
        "rsfmri_var_cortgordon_gp304rh",
        "rsfmri_var_cortgordon_gp305rh",
        "rsfmri_var_cortgordon_gp306rh",
        "rsfmri_var_cortgordon_gp307rh",
        "rsfmri_var_cortgordon_gp308rh",
        "rsfmri_var_cortgordon_gp309rh",
        "rsfmri_var_cortgordon_gp310rh",
        "rsfmri_var_cortgordon_gp311rh",
        "rsfmri_var_cortgordon_gp312rh",
        "rsfmri_var_cortgordon_gp313rh",
        "rsfmri_var_cortgordon_gp314rh",
        "rsfmri_var_cortgordon_gp315rh",
        "rsfmri_var_cortgordon_gp316rh",
        "rsfmri_var_cortgordon_gp317rh",
        "rsfmri_var_cortgordon_gp318rh",
        "rsfmri_var_cortgordon_gp319rh",
        "rsfmri_var_cortgordon_gp320rh",
        "rsfmri_var_cortgordon_gp321rh",
        "rsfmri_var_cortgordon_gp322rh",
        "rsfmri_var_cortgordon_gp323rh",
        "rsfmri_var_cortgordon_gp324rh",
        "rsfmri_var_cortgordon_gp325rh",
        "rsfmri_var_cortgordon_gp326rh",
        "rsfmri_var_cortgordon_gp327rh",
        "rsfmri_var_cortgordon_gp328rh",
        "rsfmri_var_cortgordon_gp329rh",
        "rsfmri_var_cortgordon_gp330rh",
        "rsfmri_var_cortgordon_gp331rh",
        "rsfmri_var_cortgordon_gp332rh",
        "rsfmri_var_cortgordon_gp333rh",
        "rsfmri_var_scs_crbcortexlh",
        "rsfmri_var_scs_tplh",
        "rsfmri_var_scs_caudatelh",
        "rsfmri_var_scs_putamenlh",
        "rsfmri_var_scs_pallidumlh",
        "rsfmri_var_scs_brainstem",
        "rsfmri_var_scs_hpuslh",
        "rsfmri_var_scs_amygdalalh",
        "rsfmri_var_scs_aalh",
        "rsfmri_var_scs_ventraldclh",
        "rsfmri_var_scs_crbcortexrh",
        "rsfmri_var_scs_tprh",
        "rsfmri_var_scs_caudaterh",
        "rsfmri_var_scs_putamenrh",
        "rsfmri_var_scs_pallidumrh",
        "rsfmri_var_scs_hpusrh",
        "rsfmri_var_scs_amygdalarh",
        "rsfmri_var_scs_lesionrh",
        "rsfmri_var_scs_aarh",
        "rsfmri_var_scs_ventraldcrh",
    ],
    "abcd_betnet02": [
        "rsfmri_c_ngd_ad_ngd_ad",
        "rsfmri_c_ngd_ad_ngd_cgc",
        "rsfmri_c_ngd_ad_ngd_ca",
        "rsfmri_c_ngd_ad_ngd_dt",
        "rsfmri_c_ngd_ad_ngd_dla",
        "rsfmri_c_ngd_ad_ngd_fo",
        "rsfmri_c_ngd_ad_ngd_n",
        "rsfmri_c_ngd_ad_ngd_rspltp",
        "rsfmri_c_ngd_ad_ngd_smh",
        "rsfmri_c_ngd_ad_ngd_smm",
        "rsfmri_c_ngd_ad_ngd_sa",
        "rsfmri_c_ngd_ad_ngd_vta",
        "rsfmri_c_ngd_ad_ngd_vs",
        "rsfmri_c_ngd_cgc_ngd_cgc",
        "rsfmri_c_ngd_cgc_ngd_ca",
        "rsfmri_c_ngd_cgc_ngd_dt",
        "rsfmri_c_ngd_cgc_ngd_dla",
        "rsfmri_c_ngd_cgc_ngd_fo",
        "rsfmri_c_ngd_cgc_ngd_n",
        "rsfmri_c_ngd_cgc_ngd_rspltp",
        "rsfmri_c_ngd_cgc_ngd_smh",
        "rsfmri_c_ngd_cgc_ngd_smm",
        "rsfmri_c_ngd_cgc_ngd_sa",
        "rsfmri_c_ngd_cgc_ngd_vta",
        "rsfmri_c_ngd_cgc_ngd_vs",
        "rsfmri_c_ngd_ca_ngd_ca",
        "rsfmri_c_ngd_ca_ngd_dt",
        "rsfmri_c_ngd_ca_ngd_dla",
        "rsfmri_c_ngd_ca_ngd_fo",
        "rsfmri_c_ngd_ca_ngd_n",
        "rsfmri_c_ngd_ca_ngd_rspltp",
        "rsfmri_c_ngd_ca_ngd_smh",
        "rsfmri_c_ngd_ca_ngd_smm",
        "rsfmri_c_ngd_ca_ngd_sa",
        "rsfmri_c_ngd_ca_ngd_vta",
        "rsfmri_c_ngd_ca_ngd_vs",
        "rsfmri_c_ngd_dt_ngd_dt",
        "rsfmri_c_ngd_dt_ngd_dla",
        "rsfmri_c_ngd_dt_ngd_fo",
        "rsfmri_c_ngd_dt_ngd_n",
        "rsfmri_c_ngd_dt_ngd_rspltp",
        "rsfmri_c_ngd_dt_ngd_smh",
        "rsfmri_c_ngd_dt_ngd_smm",
        "rsfmri_c_ngd_dt_ngd_sa",
        "rsfmri_c_ngd_dt_ngd_vta",
        "rsfmri_c_ngd_dt_ngd_vs",
        "rsfmri_c_ngd_dla_ngd_dla",
        "rsfmri_c_ngd_dla_ngd_fo",
        "rsfmri_c_ngd_dla_ngd_n",
        "rsfmri_c_ngd_dla_ngd_rspltp",
        "rsfmri_c_ngd_dla_ngd_smh",
        "rsfmri_c_ngd_dla_ngd_smm",
        "rsfmri_c_ngd_dla_ngd_sa",
        "rsfmri_c_ngd_dla_ngd_vta",
        "rsfmri_c_ngd_dla_ngd_vs",
        "rsfmri_c_ngd_fo_ngd_fo",
        "rsfmri_c_ngd_fo_ngd_n",
        "rsfmri_c_ngd_fo_ngd_rspltp",
        "rsfmri_c_ngd_fo_ngd_smh",
        "rsfmri_c_ngd_fo_ngd_smm",
        "rsfmri_c_ngd_fo_ngd_sa",
        "rsfmri_c_ngd_fo_ngd_vta",
        "rsfmri_c_ngd_fo_ngd_vs",
        "rsfmri_c_ngd_n_ngd_n",
        "rsfmri_c_ngd_n_ngd_rspltp",
        "rsfmri_c_ngd_n_ngd_smh",
        "rsfmri_c_ngd_n_ngd_smm",
        "rsfmri_c_ngd_n_ngd_sa",
        "rsfmri_c_ngd_n_ngd_vta",
        "rsfmri_c_ngd_n_ngd_vs",
        "rsfmri_c_ngd_rspltp_ngd_rspltp",
        "rsfmri_c_ngd_rspltp_ngd_smh",
        "rsfmri_c_ngd_rspltp_ngd_smm",
        "rsfmri_c_ngd_rspltp_ngd_sa",
        "rsfmri_c_ngd_rspltp_ngd_vta",
        "rsfmri_c_ngd_rspltp_ngd_vs",
        "rsfmri_c_ngd_smh_ngd_smh",
        "rsfmri_c_ngd_smh_ngd_smm",
        "rsfmri_c_ngd_smh_ngd_sa",
        "rsfmri_c_ngd_smh_ngd_vta",
        "rsfmri_c_ngd_smh_ngd_vs",
        "rsfmri_c_ngd_smm_ngd_smm",
        "rsfmri_c_ngd_smm_ngd_sa",
        "rsfmri_c_ngd_smm_ngd_vta",
        "rsfmri_c_ngd_smm_ngd_vs",
        "rsfmri_c_ngd_sa_ngd_sa",
        "rsfmri_c_ngd_sa_ngd_vta",
        "rsfmri_c_ngd_sa_ngd_vs",
        "rsfmri_c_ngd_vta_ngd_vta",
        "rsfmri_c_ngd_vta_ngd_vs",
        "rsfmri_c_ngd_vs_ngd_vs",
    ],
    "mrirscor02": [
        "rsfmri_cor_ngd_au_scs_crcxlh",
        "rsfmri_cor_ngd_au_scs_thplh",
        "rsfmri_cor_ngd_au_scs_cdelh",
        "rsfmri_cor_ngd_au_scs_ptlh",
        "rsfmri_cor_ngd_au_scs_pllh",
        "rsfmri_cor_ngd_au_scs_bs",
        "rsfmri_cor_ngd_au_scs_hplh",
        "rsfmri_cor_ngd_au_scs_aglh",
        "rsfmri_cor_ngd_au_scs_aalh",
        "rsfmri_cor_ngd_au_scs_vtdclh",
        "rsfmri_cor_ngd_au_scs_crcxrh",
        "rsfmri_cor_ngd_au_scs_thprh",
        "rsfmri_cor_ngd_au_scs_cderh",
        "rsfmri_cor_ngd_au_scs_ptrh",
        "rsfmri_cor_ngd_au_scs_plrh",
        "rsfmri_cor_ngd_au_scs_hprh",
        "rsfmri_cor_ngd_au_scs_agrh",
        "rsfmri_cor_ngd_au_scs_aarh",
        "rsfmri_cor_ngd_au_scs_vtdcrh",
        "rsfmri_cor_ngd_cerc_scs_crcxlh",
        "rsfmri_cor_ngd_cerc_scs_thplh",
        "rsfmri_cor_ngd_cerc_scs_cdelh",
        "rsfmri_cor_ngd_cerc_scs_ptlh",
        "rsfmri_cor_ngd_cerc_scs_pllh",
        "rsfmri_cor_ngd_cerc_scs_bs",
        "rsfmri_cor_ngd_cerc_scs_hplh",
        "rsfmri_cor_ngd_cerc_scs_aglh",
        "rsfmri_cor_ngd_cerc_scs_aalh",
        "rsfmri_cor_ngd_cerc_scs_vtdclh",
        "rsfmri_cor_ngd_cerc_scs_crcxrh",
        "rsfmri_cor_ngd_cerc_scs_thprh",
        "rsfmri_cor_ngd_cerc_scs_cderh",
        "rsfmri_cor_ngd_cerc_scs_ptrh",
        "rsfmri_cor_ngd_cerc_scs_plrh",
        "rsfmri_cor_ngd_cerc_scs_hprh",
        "rsfmri_cor_ngd_cerc_scs_agrh",
        "rsfmri_cor_ngd_cerc_scs_aarh",
        "rsfmri_cor_ngd_cerc_scs_vtdcrh",
        "rsfmri_cor_ngd_copa_scs_crcxlh",
        "rsfmri_cor_ngd_copa_scs_thplh",
        "rsfmri_cor_ngd_copa_scs_cdelh",
        "rsfmri_cor_ngd_copa_scs_ptlh",
        "rsfmri_cor_ngd_copa_scs_pllh",
        "rsfmri_cor_ngd_copa_scs_bs",
        "rsfmri_cor_ngd_copa_scs_hplh",
        "rsfmri_cor_ngd_copa_scs_aglh",
        "rsfmri_cor_ngd_copa_scs_aalh",
        "rsfmri_cor_ngd_copa_scs_vtdclh",
        "rsfmri_cor_ngd_copa_scs_crcxrh",
        "rsfmri_cor_ngd_copa_scs_thprh",
        "rsfmri_cor_ngd_copa_scs_cderh",
        "rsfmri_cor_ngd_copa_scs_ptrh",
        "rsfmri_cor_ngd_copa_scs_plrh",
        "rsfmri_cor_ngd_copa_scs_hprh",
        "rsfmri_cor_ngd_copa_scs_agrh",
        "rsfmri_cor_ngd_copa_scs_aarh",
        "rsfmri_cor_ngd_copa_scs_vtdcrh",
        "rsfmri_cor_ngd_df_scs_crcxlh",
        "rsfmri_cor_ngd_df_scs_thplh",
        "rsfmri_cor_ngd_df_scs_cdelh",
        "rsfmri_cor_ngd_df_scs_ptlh",
        "rsfmri_cor_ngd_df_scs_pllh",
        "rsfmri_cor_ngd_df_scs_bs",
        "rsfmri_cor_ngd_df_scs_hplh",
        "rsfmri_cor_ngd_df_scs_aglh",
        "rsfmri_cor_ngd_df_scs_aalh",
        "rsfmri_cor_ngd_df_scs_vtdclh",
        "rsfmri_cor_ngd_df_scs_crcxrh",
        "rsfmri_cor_ngd_df_scs_thprh",
        "rsfmri_cor_ngd_df_scs_cderh",
        "rsfmri_cor_ngd_df_scs_ptrh",
        "rsfmri_cor_ngd_df_scs_plrh",
        "rsfmri_cor_ngd_df_scs_hprh",
        "rsfmri_cor_ngd_df_scs_agrh",
        "rsfmri_cor_ngd_df_scs_aarh",
        "rsfmri_cor_ngd_df_scs_vtdcrh",
        "rsfmri_cor_ngd_dsa_scs_crcxlh",
        "rsfmri_cor_ngd_dsa_scs_thplh",
        "rsfmri_cor_ngd_dsa_scs_cdelh",
        "rsfmri_cor_ngd_dsa_scs_ptlh",
        "rsfmri_cor_ngd_dsa_scs_pllh",
        "rsfmri_cor_ngd_dsa_scs_bs",
        "rsfmri_cor_ngd_dsa_scs_hplh",
        "rsfmri_cor_ngd_dsa_scs_aglh",
        "rsfmri_cor_ngd_dsa_scs_aalh",
        "rsfmri_cor_ngd_dsa_scs_vtdclh",
        "rsfmri_cor_ngd_dsa_scs_crcxrh",
        "rsfmri_cor_ngd_dsa_scs_thprh",
        "rsfmri_cor_ngd_dsa_scs_cderh",
        "rsfmri_cor_ngd_dsa_scs_ptrh",
        "rsfmri_cor_ngd_dsa_scs_plrh",
        "rsfmri_cor_ngd_dsa_scs_hprh",
        "rsfmri_cor_ngd_dsa_scs_agrh",
        "rsfmri_cor_ngd_dsa_scs_aarh",
        "rsfmri_cor_ngd_dsa_scs_vtdcrh",
        "rsfmri_cor_ngd_fopa_scs_crcxlh",
        "rsfmri_cor_ngd_fopa_scs_thplh",
        "rsfmri_cor_ngd_fopa_scs_cdelh",
        "rsfmri_cor_ngd_fopa_scs_ptlh",
        "rsfmri_cor_ngd_fopa_scs_pllh",
        "rsfmri_cor_ngd_fopa_scs_bs",
        "rsfmri_cor_ngd_fopa_scs_hplh",
        "rsfmri_cor_ngd_fopa_scs_aglh",
        "rsfmri_cor_ngd_fopa_scs_aalh",
        "rsfmri_cor_ngd_fopa_scs_vtdclh",
        "rsfmri_cor_ngd_fopa_scs_crcxrh",
        "rsfmri_cor_ngd_fopa_scs_thprh",
        "rsfmri_cor_ngd_fopa_scs_cderh",
        "rsfmri_cor_ngd_fopa_scs_ptrh",
        "rsfmri_cor_ngd_fopa_scs_plrh",
        "rsfmri_cor_ngd_fopa_scs_hprh",
        "rsfmri_cor_ngd_fopa_scs_agrh",
        "rsfmri_cor_ngd_fopa_scs_aarh",
        "rsfmri_cor_ngd_fopa_scs_vtdcrh",
        "rsfmri_cor_ngd_none_scs_crcxlh",
        "rsfmri_cor_ngd_none_scs_thplh",
        "rsfmri_cor_ngd_none_scs_cdelh",
        "rsfmri_cor_ngd_none_scs_ptlh",
        "rsfmri_cor_ngd_none_scs_pllh",
        "rsfmri_cor_ngd_none_scs_bs",
        "rsfmri_cor_ngd_none_scs_hplh",
        "rsfmri_cor_ngd_none_scs_aglh",
        "rsfmri_cor_ngd_none_scs_aalh",
        "rsfmri_cor_ngd_none_scs_vtdclh",
        "rsfmri_cor_ngd_none_scs_crcxrh",
        "rsfmri_cor_ngd_none_scs_thprh",
        "rsfmri_cor_ngd_none_scs_cderh",
        "rsfmri_cor_ngd_none_scs_ptrh",
        "rsfmri_cor_ngd_none_scs_plrh",
        "rsfmri_cor_ngd_none_scs_hprh",
        "rsfmri_cor_ngd_none_scs_agrh",
        "rsfmri_cor_ngd_none_scs_aarh",
        "rsfmri_cor_ngd_none_scs_vtdcrh",
        "rsfmri_cor_ngd_rst_scs_crcxlh",
        "rsfmri_cor_ngd_rst_scs_thplh",
        "rsfmri_cor_ngd_rst_scs_cdelh",
        "rsfmri_cor_ngd_rst_scs_ptlh",
        "rsfmri_cor_ngd_rst_scs_pllh",
        "rsfmri_cor_ngd_rst_scs_bs",
        "rsfmri_cor_ngd_rst_scs_hplh",
        "rsfmri_cor_ngd_rst_scs_aglh",
        "rsfmri_cor_ngd_rst_scs_aalh",
        "rsfmri_cor_ngd_rst_scs_vtdclh",
        "rsfmri_cor_ngd_rst_scs_crcxrh",
        "rsfmri_cor_ngd_rst_scs_thprh",
        "rsfmri_cor_ngd_rst_scs_cderh",
        "rsfmri_cor_ngd_rst_scs_ptrh",
        "rsfmri_cor_ngd_rst_scs_plrh",
        "rsfmri_cor_ngd_rst_scs_hprh",
        "rsfmri_cor_ngd_rst_scs_agrh",
        "rsfmri_cor_ngd_rst_scs_aarh",
        "rsfmri_cor_ngd_rst_scs_vtdcrh",
        "rsfmri_cor_ngd_smh_scs_crcxlh",
        "rsfmri_cor_ngd_smh_scs_thplh",
        "rsfmri_cor_ngd_smh_scs_cdelh",
        "rsfmri_cor_ngd_smh_scs_ptlh",
        "rsfmri_cor_ngd_smh_scs_pllh",
        "rsfmri_cor_ngd_smh_scs_bs",
        "rsfmri_cor_ngd_smh_scs_hplh",
        "rsfmri_cor_ngd_smh_scs_aglh",
        "rsfmri_cor_ngd_smh_scs_aalh",
        "rsfmri_cor_ngd_smh_scs_vtdclh",
        "rsfmri_cor_ngd_smh_scs_crcxrh",
        "rsfmri_cor_ngd_smh_scs_thprh",
        "rsfmri_cor_ngd_smh_scs_cderh",
        "rsfmri_cor_ngd_smh_scs_ptrh",
        "rsfmri_cor_ngd_smh_scs_plrh",
        "rsfmri_cor_ngd_smh_scs_hprh",
        "rsfmri_cor_ngd_smh_scs_agrh",
        "rsfmri_cor_ngd_smh_scs_aarh",
        "rsfmri_cor_ngd_smh_scs_vtdcrh",
        "rsfmri_cor_ngd_smm_scs_crcxlh",
        "rsfmri_cor_ngd_smm_scs_thplh",
        "rsfmri_cor_ngd_smm_scs_cdelh",
        "rsfmri_cor_ngd_smm_scs_ptlh",
        "rsfmri_cor_ngd_smm_scs_pllh",
        "rsfmri_cor_ngd_smm_scs_bs",
        "rsfmri_cor_ngd_smm_scs_hplh",
        "rsfmri_cor_ngd_smm_scs_aglh",
        "rsfmri_cor_ngd_smm_scs_aalh",
        "rsfmri_cor_ngd_smm_scs_vtdclh",
        "rsfmri_cor_ngd_smm_scs_crcxrh",
        "rsfmri_cor_ngd_smm_scs_thprh",
        "rsfmri_cor_ngd_smm_scs_cderh",
        "rsfmri_cor_ngd_smm_scs_ptrh",
        "rsfmri_cor_ngd_smm_scs_plrh",
        "rsfmri_cor_ngd_smm_scs_hprh",
        "rsfmri_cor_ngd_smm_scs_agrh",
        "rsfmri_cor_ngd_smm_scs_aarh",
        "rsfmri_cor_ngd_smm_scs_vtdcrh",
        "rsfmri_cor_ngd_sa_scs_crcxlh",
        "rsfmri_cor_ngd_sa_scs_thplh",
        "rsfmri_cor_ngd_sa_scs_cdelh",
        "rsfmri_cor_ngd_sa_scs_ptlh",
        "rsfmri_cor_ngd_sa_scs_pllh",
        "rsfmri_cor_ngd_sa_scs_bs",
        "rsfmri_cor_ngd_sa_scs_hplh",
        "rsfmri_cor_ngd_sa_scs_aglh",
        "rsfmri_cor_ngd_sa_scs_aalh",
        "rsfmri_cor_ngd_sa_scs_vtdclh",
        "rsfmri_cor_ngd_sa_scs_crcxrh",
        "rsfmri_cor_ngd_sa_scs_thprh",
        "rsfmri_cor_ngd_sa_scs_cderh",
        "rsfmri_cor_ngd_sa_scs_ptrh",
        "rsfmri_cor_ngd_sa_scs_plrh",
        "rsfmri_cor_ngd_sa_scs_hprh",
        "rsfmri_cor_ngd_sa_scs_agrh",
        "rsfmri_cor_ngd_sa_scs_aarh",
        "rsfmri_cor_ngd_sa_scs_vtdcrh",
        "rsfmri_cor_ngd_vta_scs_crcxlh",
        "rsfmri_cor_ngd_vta_scs_thplh",
        "rsfmri_cor_ngd_vta_scs_cdelh",
        "rsfmri_cor_ngd_vta_scs_ptlh",
        "rsfmri_cor_ngd_vta_scs_pllh",
        "rsfmri_cor_ngd_vta_scs_bs",
        "rsfmri_cor_ngd_vta_scs_hplh",
        "rsfmri_cor_ngd_vta_scs_aglh",
        "rsfmri_cor_ngd_vta_scs_aalh",
        "rsfmri_cor_ngd_vta_scs_vtdclh",
        "rsfmri_cor_ngd_vta_scs_crcxrh",
        "rsfmri_cor_ngd_vta_scs_thprh",
        "rsfmri_cor_ngd_vta_scs_cderh",
        "rsfmri_cor_ngd_vta_scs_ptrh",
        "rsfmri_cor_ngd_vta_scs_plrh",
        "rsfmri_cor_ngd_vta_scs_hprh",
        "rsfmri_cor_ngd_vta_scs_agrh",
        "rsfmri_cor_ngd_vta_scs_aarh",
        "rsfmri_cor_ngd_vta_scs_vtdcrh",
        "rsfmri_cor_ngd_vs_scs_crcxlh",
        "rsfmri_cor_ngd_vs_scs_thplh",
        "rsfmri_cor_ngd_vs_scs_cdelh",
        "rsfmri_cor_ngd_vs_scs_ptlh",
        "rsfmri_cor_ngd_vs_scs_pllh",
        "rsfmri_cor_ngd_vs_scs_bs",
        "rsfmri_cor_ngd_vs_scs_hplh",
        "rsfmri_cor_ngd_vs_scs_aglh",
        "rsfmri_cor_ngd_vs_scs_aalh",
        "rsfmri_cor_ngd_vs_scs_vtdclh",
        "rsfmri_cor_ngd_vs_scs_crcxrh",
        "rsfmri_cor_ngd_vs_scs_thprh",
        "rsfmri_cor_ngd_vs_scs_cderh",
        "rsfmri_cor_ngd_vs_scs_ptrh",
        "rsfmri_cor_ngd_vs_scs_plrh",
        "rsfmri_cor_ngd_vs_scs_hprh",
        "rsfmri_cor_ngd_vs_scs_agrh",
        "rsfmri_cor_ngd_vs_scs_aarh",
        "rsfmri_cor_ngd_vs_scs_vtdcrh",
    ],
    "abcd_tbss01": [
        "nihtbx_picvocab_uncorrected",
        "nihtbx_flanker_uncorrected",
        "nihtbx_list_uncorrected",
        "nihtbx_cardsort_uncorrected",
        "nihtbx_pattern_uncorrected",
        "nihtbx_picture_uncorrected",
        "nihtbx_reading_uncorrected",
    ],
    "abcd_mri01": [
        "mri_info_manufacturer",
        "interview_date"
    ],
    "acspsw03": [
        "rel_family_id", 
        "rel_group_id", 
        "rel_ingroup_order", 
        "rel_relationship",
        "race_ethnicity"
    ],
    "abcd_lt01": ["site_id_l"],
    "pdem02": [#"demo_prnt_ethn_v2",
        "demo_prnt_marital_v2",
        "demo_prnt_ed_v2",
        "demo_comb_income_v2",
        #"demo_race_a_p___10",
        #"demo_race_a_p___11",
        #"demo_race_a_p___12",
        #"demo_race_a_p___13",
        #"demo_race_a_p___14",
        #"demo_race_a_p___15",
        #"demo_race_a_p___16",
        #"demo_race_a_p___17",
        #"demo_race_a_p___18",
        #"demo_race_a_p___19",
        #"demo_race_a_p___20",
        #"demo_race_a_p___21",
        #"demo_race_a_p___22",
        #"demo_race_a_p___23",
        #"demo_race_a_p___24",
        #"demo_race_a_p___25",
    ],
    "abcd_ssphp01": ["pds_p_ss_female_category_2",
                     "pds_p_ss_male_category_2", 
                     "sex"]
}

In [3]:
print(variables.keys())

dict_keys(['abcd_smrip10201', 'abcd_smrip20201', 'abcd_smrip30201', 'abcd_mrisdp10201', 'abcd_mrisdp20201', 'abcd_dti_p101', 'abcd_drsip101', 'abcd_drsip201', 'abcd_mrirsfd01', 'abcd_mrirstv02', 'abcd_betnet02', 'mrirscor02', 'abcd_tbss01', 'abcd_mri01', 'acspsw03', 'abcd_lt01', 'pdem02', 'abcd_ssphp01'])


In [4]:
# read in csvs of interest one a time so you don't crash your computer
# grab the vars you want, then clear the rest and read in the next
# make one "missing" column for each modality if, like RSI, a subj is missing
# on all vals if missing on one. double check this.
# also include qa column per modality and make missingness chart before/after data censoring

data_dir = (
    "/Volumes/projects_herting/LABDOCS/PROJECTS/ABCD/Data/release4.0/"
)

changes = ['abcd_smrip10201', 'abcd_smrip20201', 'abcd_smrip30201', 
           'abcd_mrisdp10201', 'abcd_mrisdp20201', 'abcd_dti_p101', 
           'abcd_drsip101', 'abcd_drsip201', 'abcd_mrirsfd01', 
           'abcd_mrirstv02', 'abcd_betnet02', 'mrirscor02', 'abcd_tbss01']

In [5]:
# column names to retain in the mega dataset
qc_vars = [
    "imgincl_dmri_include",
    "imgincl_rsfmri_include",
    "rsfmri_c_ngd_meanmotion",
    "rsfmri_c_ngd_ntpoints",
    "imgincl_t1w_include",
    "imgincl_t2w_include",
    "interview_age",
]

In [6]:
# initialize the progress bars
manager = enlighten.get_manager()
tocks = manager.counter(total=len(variables.keys()), desc='Data Structures', unit='data structures')

# keep track of variables that don't make it into the big df
missing = {}

# build the mega_df now
df = pd.DataFrame()
for key in variables.keys():
    missing[key] = []
    old_columns = len(df.columns)
    if key in changes:
        path = join(data_dir, 'change_scores', f'{key}_changescores_bl_tp2.csv')
        if exists(path):
            ticks = manager.counter(total=len(variables[key]), desc=key, unit='variables')
            temp_df = pd.read_csv(path, index_col="subjectkey", header=0)
            for column in variables[key]:
                try:
                    df[f'{column}.baseline_year_1_arm_1'] = temp_df[f'{column}.baseline_year_1_arm_1'].copy()
                    df[f'{column}.2_year_follow_up_y_arm_1'] = temp_df[f'{column}.2_year_follow_up_y_arm_1'].copy()
                    if column not in qc_vars:
                        df[f'{column}.change_score'] = temp_df[f'{column}.change_score'].copy()
                    else:
                        pass
                    if 'rsfmri_c' in column:
                        df[f'{column}.change_score'] = temp_df[f'{column}.change_score'].copy()
                        df[f'{column}.sign_change'] = temp_df[f'{column}.sign_change'].copy()
                    else:
                        pass
                except:
                    missing[key].append(column)
                ticks.update()
            ticks.refresh()
        else:
            print(f'There\'s something wrong with your path for {key}:\n{path}')
            break
    else:
        path = join(data_dir, 'csv', f'{key}.csv')
        if exists(path):
            ticks = manager.counter(total=len(variables[key]), desc=key, unit='variables')
            temp_df = pd.read_csv(path, index_col="subjectkey", header=0, skiprows=[1])
            base_df = temp_df[temp_df["eventname"] == "baseline_year_1_arm_1"].copy()
            y2fu_df = temp_df[temp_df["eventname"] == "2_year_follow_up_y_arm_1"].copy()
            temp_df = None
            for column in variables[key]:
                df[f'{column}.baseline_year_1_arm_1'] = base_df[column].copy()
                df[f'{column}.2_year_follow_up_y_arm_1'] = y2fu_df[column].copy()
                ticks.update()
            ticks.refresh()
    new_columns = len(df.columns) - old_columns
    print(f"\t{new_columns} variables added!")
    temp_df = None
    base_df = None
    y2fu_df = None
    tocks.update()

/var/folders/t5/4bz0680d45n5_5m30ny85m540000gp/T/ipykernel_1815/104078155.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}.baseline_year_1_arm_1'] = temp_df[f'{column}.baseline_year_1_arm_1'].copy()
/var/folders/t5/4bz0680d45n5_5m30ny85m540000gp/T/ipykernel_1815/104078155.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}.2_year_follow_up_y_arm_1'] = temp_df[f'{column}.2_year_follow_up_y_arm_1'].copy()
/var/folders/t5/4bz0680d45n5_5m30ny85m540000gp/T/ipykernel_1815/104078155.py:23: Performance

	631 variables added!
	339 variables added!
	63 variables added!
	888 variables added!
	444 variables added!
	333 variables added!
	186 variables added!
	180 variables added!
	444 variables added!
	1263 variables added!


/var/folders/t5/4bz0680d45n5_5m30ny85m540000gp/T/ipykernel_1815/104078155.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}.sign_change'] = temp_df[f'{column}.sign_change'].copy()


	364 variables added!
	988 variables added!
	21 variables added!


/var/folders/t5/4bz0680d45n5_5m30ny85m540000gp/T/ipykernel_1815/104078155.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}.baseline_year_1_arm_1'] = base_df[column].copy()
/var/folders/t5/4bz0680d45n5_5m30ny85m540000gp/T/ipykernel_1815/104078155.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}.2_year_follow_up_y_arm_1'] = y2fu_df[column].copy()


	4 variables added!
	10 variables added!
	2 variables added!
	6 variables added!
	6 variables added!


In [7]:
print(f"Full dataframe is {sys.getsizeof(df) / 1000000000}GB.")

Full dataframe is 0.586442498GB.


In [8]:
df.to_csv(
    "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/data/data.csv"
)

In [13]:
df.filter(regex=".*sign_change").describe()

,rsfmri_c_ngd_ad_ngd_ad.sign_change,rsfmri_c_ngd_ad_ngd_cgc.sign_change,rsfmri_c_ngd_ad_ngd_ca.sign_change,rsfmri_c_ngd_ad_ngd_dt.sign_change,rsfmri_c_ngd_ad_ngd_dla.sign_change,rsfmri_c_ngd_ad_ngd_fo.sign_change,rsfmri_c_ngd_ad_ngd_n.sign_change,rsfmri_c_ngd_ad_ngd_rspltp.sign_change,rsfmri_c_ngd_ad_ngd_smh.sign_change,rsfmri_c_ngd_ad_ngd_smm.sign_change,...,rsfmri_cor_ngd_vs_scs_vtdclh.sign_change,rsfmri_cor_ngd_vs_scs_crcxrh.sign_change,rsfmri_cor_ngd_vs_scs_thprh.sign_change,rsfmri_cor_ngd_vs_scs_cderh.sign_change,rsfmri_cor_ngd_vs_scs_ptrh.sign_change,rsfmri_cor_ngd_vs_scs_plrh.sign_change,rsfmri_cor_ngd_vs_scs_hprh.sign_change,rsfmri_cor_ngd_vs_scs_agrh.sign_change,rsfmri_cor_ngd_vs_scs_aarh.sign_change,rsfmri_cor_ngd_vs_scs_vtdcrh.sign_change
count,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,...,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000,7375.000000
mean,0.995797,0.995661,0.995797,0.995254,0.995525,0.995390,0.993627,0.995797,0.992136,0.995797,...,0.995254,0.995525,0.995390,0.993627,0.995797,0.992136,0.995797,0.995797,0.995390,0.995797
std,0.173068,0.181853,0.173068,0.180720,0.176936,0.176549,0.209841,0.173068,0.213951,0.173068,...,0.180720,0.176936,0.176549,0.209841,0.173068,0.213951,0.173068,0.173068,0.181099,0.173068
min,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


NameError: name 'df' is not defined